先上论文：https://arxiv.org/pdf/1803.05170.pdf  
git: https://github.com/Leavingseason/xDeepFM/blob/master/exdeepfm/src/exDeepFM.py  

In [ ]:
import  tensorflow  as tf
import numpy as np
arr1 = tf.convert_to_tensor(np.arange(1,25).reshape(2,4,3),dtype=tf.int32)
arr2 = tf.convert_to_tensor(np.arange(1,25).reshape(2,4,3),dtype=tf.int32)

In [ ]:
arr1

In [ ]:
split_arr1 = tf.split(arr1,[1,1,1],2)#3个（2，4，1）
split_arr2 = tf.split(arr2,[1,1,1],2)

In [ ]:
#split_arr1, split_arr2
tf.matmul(split_arr1[0],split_arr2[0],transpose_b=True)

In [ ]:
res = tf.matmul(split_arr1,split_arr2,transpose_b=True)#（3，2，4，4）

In [ ]:
res = tf.transpose(res,perm=[1,0,2,3])

In [ ]:
res

In [1]:
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras import backend as K
from tensorflow.keras.layers import Layer
from tensorflow.keras.regularizers import l2

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

/home/baogong/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import sys
sys.path.append('./util/')
from utils import load_data_embdding 

users, movies, ratings = load_data_embdding()

In [3]:
data1 = pd.merge(ratings.drop(columns = ['timestamp'],axis = 1), movies, how = 'left', on = 'movieid')
data = pd.merge(data1, users, how = 'left', on = 'userid')

X = data.drop(columns = ['userid', 'movieid', 'title', 'rating'])
Y = data['rating'].values
#genres, gender, age, occupationid四个需要embedding的特征，可以分别emb也可以合并以后emb，这里分别做emb
set_genres = []
for i in movies.index:
    set_genres += movies['genres'].iloc[i]
set_genres = list(set(set_genres))
dic_genres = dict([(j, i) for i,j in enumerate(set_genres)])
dic_genres['UNK'] = len(dic_genres)
X['genres'] = X['genres'].apply(lambda x: [dic_genres[i] for i in x])
x_genres = tf.keras.preprocessing.sequence.pad_sequences(list(X['genres'].values),
                                                        value = dic_genres['UNK'],
                                                        padding = 'post',
                                                        maxlen = 6)


dic_gender = {'F':0, 'M':1}
X['gender'] = X['gender'].apply(lambda x: [dic_gender[i] for i in x])
dic_age = {1:0, 56:1, 25:2, 45:3, 50:4, 35:5, 18:6}
X['age'] = X['age'].apply(lambda x: [dic_age[x]])
list_occ = list(pd.unique(data['occupationid']))
dic_occ = dict([(j, i) for i,j in enumerate(list_occ)])
X['occupationid'] = X['occupationid'].apply(lambda x: [dic_occ[x]])

x_gender = list(X['gender'].values)
x_age = list(X['age'].values)
x_occupationid = list(X['occupationid'].values)


train_x_genres, test_x_genres, train_y, test_y = train_test_split(np.array(x_genres), Y, random_state=11)
train_x_gender, test_x_gender = train_test_split(np.array(x_gender), random_state=11)
train_x_age, test_x_age = train_test_split(np.array(x_age), random_state=11)
train_x_occupationid, test_x_occupationid = train_test_split(np.array(x_occupationid), random_state=11)

In [53]:
class CIN(Layer):
    def __init__(self, layer_size=(128, 128), activation='relu', direct=False, l2=1e-5, **kwargs):
        self.layer_size = layer_size
        self.direct = direct
        self.activation = activation
        self.l2 = l2
        super(CIN, self).__init__(**kwargs)

    def build(self, input_shape):

        self.field_nums = [int(input_shape[1])]
        self.filters = []
        self.bias = []
        
        for i, size in enumerate(self.layer_size):
            
            self.filters.append(self.add_weight(name='f_' + str(i),
                                                shape=[1, self.field_nums[-1] * self.field_nums[0], size],
                                                dtype=tf.float32, 
                                                initializer='glorot_uniform',
                                                regularizer=l2(self.l2)))

            self.bias.append(self.add_weight(name='bias' + str(i), shape=[size], dtype=tf.float32,
                                             initializer='Zeros'))
            
            if self.direct:
                self.field_nums.append(size)
            else:
                self.field_nums.append(size // 2)

        self.activation_layers = [keras.layers.Activation(
            self.activation) for _ in self.layer_size]

        super(CIN, self).build(input_shape)  

    def call(self, inputs, **kwargs):

        dim = int(inputs.get_shape()[-1])#16
        hidden_nn_layers = [inputs]
        final_result = []

        split_tensor0 = tf.split(hidden_nn_layers[0], dim * [1], 2)#16个（batch, 9, 1）
        for idx, layer_size in enumerate(self.layer_size):
            split_tensor = tf.split(hidden_nn_layers[-1], dim * [1], 2)

            dot_result_m = tf.matmul( split_tensor0, split_tensor, transpose_b=True)#（16,batch,9,9）

            dot_result_o = tf.reshape(
                dot_result_m, shape=[dim, -1, self.field_nums[0] * self.field_nums[idx]])

            dot_result = tf.transpose(dot_result_o, perm=[1, 0, 2])
            #print (dot_result.shape, self.filters[idx].shape, self.field_nums)
            #dot_result:[batch, dim, self.field_nums[0] * self.field_nums[idx]]
            #filters:[1, self.field_nums[-1]* self.field_nums[0], layer_size]
            #->curr_out:[batch, dim-1+1, layer_size]  dim维度是不变的
            '''
            conv1d: 
            x:[batch, in_width, in_channels]  
            filters:[filter_width, in_channels, out_channels] 
            out:[batch, in_width-filter_width+1, out_channels]  不考虑stride和padding
            '''
            curr_out = tf.nn.conv1d(dot_result, filters=self.filters[idx], stride=1, padding='VALID')

            curr_out = tf.nn.bias_add(curr_out, self.bias[idx])

            curr_out = self.activation_layers[idx](curr_out)

            curr_out = tf.transpose(curr_out, perm=[0, 2, 1])

            if not self.direct:
                if idx != len(self.layer_size) - 1:
                    next_hidden, direct_connect = tf.split(curr_out, 2 * [layer_size // 2], 1)
                else:
                    direct_connect = curr_out
                    next_hidden = 0
            else:
                direct_connect = curr_out
                next_hidden = curr_out

            final_result.append(direct_connect)
            hidden_nn_layers.append(next_hidden)

        result = tf.concat(final_result, axis=1)
        result = tf.reduce_sum(result, -1)

        return result

In [54]:
input_genres = keras.layers.Input(shape=(6,), name="genres")  
embedding_genres = keras.layers.Embedding(output_dim=16, input_dim=len(dic_genres), input_length=6)(input_genres)

input_gender = keras.layers.Input(shape=(1,), name="gender")  
embedding_gender = keras.layers.Embedding(output_dim=16, input_dim=2, input_length=1)(input_gender)

input_age = keras.layers.Input(shape=(1,), name="age")  
embedding_age = keras.layers.Embedding(output_dim=16, input_dim=7, input_length=1)(input_age)

input_occ = keras.layers.Input(shape=(1,), name="occupationid")  
embedding_occ = keras.layers.Embedding(output_dim=16, input_dim=21, input_length=1)(input_occ)

embedding_combine = keras.layers.concatenate(inputs=[embedding_genres, embedding_gender, embedding_age,
                                                    embedding_occ], axis=1)
embedding_combine1 = keras.layers.GlobalAveragePooling1D()(embedding_combine) 

In [58]:
cin_layer = CIN(layer_size=(128, 128), activation='relu', direct=False, l2=1e-5)(embedding_combine)
cin_layer = keras.layers.Dense(1)(cin_layer)
#dnn
dnn_layer = keras.layers.Dense(64, activation = 'relu')(embedding_combine1)
dnn_layer = keras.layers.BatchNormalization()(dnn_layer)
dnn_layer = keras.layers.Dense(32, activation = 'relu')(dnn_layer)
dnn_layer = keras.layers.BatchNormalization()(dnn_layer)
dnn_layer = keras.layers.Dense(1)(dnn_layer)
##deepfm
outputs = keras.layers.average([cin_layer, dnn_layer])  #因为是做回归，就用了average，分类的话得add后再接Activation

optimizer = keras.optimizers.RMSprop(learning_rate = 0.001)
model = tf.keras.Model(inputs = [input_genres, input_gender, input_age, input_occ], outputs = [outputs])

model.compile(loss='mean_squared_error',
        optimizer=optimizer,
        metrics=['mean_absolute_error', 'mean_squared_error'],
         )

(None, 16, 81) (1, 81, 128) ListWrapper([9, 64, 64])
(None, 16, 576) (1, 576, 128) ListWrapper([9, 64, 64])


In [59]:
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
checkpoint_path = "./model/deepcin.h5"
cp_callback = keras.callbacks.ModelCheckpoint(checkpoint_path,
                                              save_weights_only=True,
                                              save_best_only=True,
                                              verbose=1)

model.fit(
    [train_x_genres, train_x_gender, train_x_age, train_x_occupationid], train_y,
    epochs=100, 
    validation_data=([test_x_genres, test_x_gender, test_x_age, test_x_occupationid], test_y,),
    batch_size=256, shuffle=True,
    callbacks=[early_stopping, cp_callback]
)

Train on 750156 samples, validate on 250053 samples
Epoch 1/100
749824/750156 [============================>.] - ETA: 0s - loss: 1.2197 - mean_absolute_error: 0.8965 - mean_squared_error: 1.2189
Epoch 00001: val_loss improved from inf to 1.17173, saving model to ./model/deepcin.h5
750156/750156 [==============================] - 87s 115us/sample - loss: 1.2197 - mean_absolute_error: 0.8965 - mean_squared_error: 1.2189 - val_loss: 1.1717 - val_mean_absolute_error: 0.8846 - val_mean_squared_error: 1.1713
Epoch 2/100
749824/750156 [============================>.] - ETA: 0s - loss: 1.1621 - mean_absolute_error: 0.8774 - mean_squared_error: 1.1614
Epoch 00002: val_loss improved from 1.17173 to 1.15612, saving model to ./model/deepcin.h5
750156/750156 [==============================] - 84s 112us/sample - loss: 1.1621 - mean_absolute_error: 0.8774 - mean_squared_error: 1.1614 - val_loss: 1.1561 - val_mean_absolute_error: 0.8746 - val_mean_squared_error: 1.1554
Epoch 3/100
749824/750156 [=====